In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys

sys.path.append('/content/drive/MyDrive/colab')

In [5]:
import cv2
from Set import DisjointSet
import matplotlib.pyplot as plt
import google.colab.patches as colab
import numpy as np
from MNISTDataset import MNISTDataset

In [ ]:
mnist_train =MNISTDataset('/content/drive/MyDrive/colab/train-images.idx3-ubyte',
                          '/content/drive/MyDrive/colab/train-labels.idx1-ubyte')

In [28]:
winSize = (28,28)
blockSize = (28,28)
blockStride = (1,1)
cellSize = (14,14)
nbins = 9

HOG = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)

In [29]:
n_train =len(mnist_train)

trainImageMat =np.zeros((n_train,36),dtype =np.float32)
trainLabelMat =np.zeros((n_train),dtype =int)

In [30]:
for i in range(n_train):
  # get an item
  image ,label =mnist_train.getitem(i)

  # apply HOG Desciptor
  a_hog =HOG.compute(image)
  a_hog =a_hog.reshape(-1)
  a_hog =a_hog.astype(np.float32)
  
  # image =image.reshape(-1)
  # image.astype(np.float32)
  # trainImageMat[i] =image
  trainImageMat[i] =a_hog
  trainLabelMat[i] =label

In [31]:
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 10000, 1e-6))

In [32]:
svm.train(trainImageMat,cv2.ml.ROW_SAMPLE,trainLabelMat)

True

In [17]:
mnist_test =MNISTDataset('/content/drive/MyDrive/colab/t10k-images.idx3-ubyte',
                          '/content/drive/MyDrive/colab/t10k-labels.idx1-ubyte')

MNISTDataset.MNISTDataset

In [34]:
n_test =len(mnist_test)
n_correct =0
n_wrong =0

for i in range(n_test):
  image ,label =mnist_test.getitem(i)

  a_hog =HOG.compute(image)
  a_hog =a_hog.reshape((1,-1))
  a_hog =a_hog.astype(np.float32)
  # image =image.reshape((1,-1))
  # image =image.astype(np.float32)

  result =svm.predict(a_hog)
  result =int(result[1])

  if (result ==label):
    n_correct +=1
  else:
    n_wrong +=1

In [35]:
accuracy =n_correct /n_test
print('accuracy','{:.2f}'.format(accuracy))

accuracy 0.93
